# 1. Selección del problema

El ejercicio de esta práctica consiste en generar un modelo LLM que sea capaz de resolver un problema específico a libre elección. Es decir, en el enunciado no se especifica cual es el problema que se debe resolver sino que el programador debe elegir un problema que desea resolver y por medio de un modelo LLM generar una solución. Por ello, la primera decisión consiste en decidir el problema a resolver.

En mi caso, el problema a resolver está marcado por la cantidad de datos que se posee para realizar el fine tunning del modelo. Lo que se pretende es utilizar un modelo base disponible en HuggingFace y realizar un fine tunning para que el modelo se ajuste al problema específico a resolver. Dado que para hacer el fine tunning es necesario poseer datos específicos del problema, esto ya supone una primera limitación ya que debido al tiempo que se dispone para realizar la práctica, es más conveniente utilizar un dataset que este presente en HuggingFace. Por ejemplo en un principio había planteado  generar un asistente virtual que ayudará a futuros alumnos que desearán estudiar el bootcamp de IA a resolver todas sus dudas o inquietdues. Sin embargo, para poder hacer esto era necesario poseer un dataset que contuviera información específica de este tema y dado que no existe ninguno HuggingFace, esta idea la descarté ya generar este dataset supondría un gran coste de tiempo.

Es por eso que siguiendo esta idea, es más sencillo generar un modelo que sea posible realizarle un fine tunning en base a un dataset ya existente que crear un nuevo dataset para realizar el fine tunning. Es por eso que lo primero que hice fue buscar distintos datasets de Hugging Face que pudieran ser utilizados para crear un asistente virtual específico de un tema. Esto de primeras ya supone una limitación ya que no existen datasets en Hugging Face de cualquier tema. Además, a la hora de crear un chatbot hay que tener en cuenta las consideraciones éticas y legales ya que por ejemplo había planteado generar un chatbot centrado en el Covid-19, que ayudara a los pacientes que sufren síntomas a resolver sus dudas e inquietdudes pero investigando un poco en las consecuencias que puede tener a nivel legal y ético, creo que es más conveniente hacer un chatbot sobre un tema que no sea tan controvertido.

Por ello, estuve un tiempo investigando los distintos datasets presentes en Hugging Face y estuve leyendo acerca de datasets utilizados para realizar fine tunning, y observé que mucha gente recomendaba utilizar el dataset squad. Este dataset está compuesto por cinco columnas y su principal función es en base a un contexto, resolver una pregunta que se plantea. Por ejemplo, se le introduce al modelo un texto y por medio de una pregunta, el modelo debe de intentar dar la misma respuesta que la indicada en la columna answer. Una ventaja que tiene este dataset es que contiene muchas filas (98169 en total) lo cual es muy bueno ya que permite que el modeo pueda aprender más. Además su formato estructurado en contexto-pregunta-respuesta creo que es bueno ya que creo que esto puede hacer más fácil realizar el fine-tunning. Es por eso que me decanté por esta opción y decidí generar un chatbot que en base a un contexto, el modelo fuera capaz de responder preguntas.

Elegido el dataset, comencé a investigar modelo bases que podía utilizar y tras realizar un estudio decidí decantarme por el modelo DistilBERT por varios motivos:

* En primer lugar, porque proviene del modelo BERT pero en una versión más ligera y rápida lo cual creo que es conveniente dado la falta de recursos computacionales que tengo.

* Por otro lado, el hecho de que provenga del modelo BERT creo que es adecuado ya que el modelo BERT ha sido entrenado con muchos textos lo cual lo hace adecuado para tareas de NLP como es la generación de respuestas en base a un contexto dado.

* Por último, tanto el modelo como el dataset se encuentran en Hugging Face lo cual permite utilizar la biblioteca transformers para cargar todo. Esto permite que sea más flexible el problema.


Por todo ello, el problema que se va a plantear en este ejercicio es el de crear un asistente virtual que dado un contexto sea capaz de responder preguntas. Para ello, se va a utilizar como modelo base el modelo DistilBERT y para ajustarlo se va a utilizar el dataset SQuAD.

# 2. Carga del modelo y dataset

Comienzo el ejercicio configurando el entorno e importando las librerías.

In [2]:
# Librerías necesarias
!pip install transformers datasets
import torch
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

Asimismo, descargo la última actualización de la librería transformers torch.

In [3]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Importo el dataset, su tokenizador y el modelo base. El tokenizador lo importo porque aunque se pueda realizar la tokenización con otras técnicas, siempre es conveniente utilizar el tokenizador del propio modelo base.

In [ ]:
# Modelo base y el tokenizador
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

# Dataset SQuAD
dataset = load_dataset("squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Importado el dataset, procedo a investigarlo un poco.

In [ ]:
dataset.shape

{'train': (87599, 5), 'validation': (10570, 5)}

Lo primero que podemos observar es que el dataset está dividido en dos: train y validation con una proporción de 88-12 por ciento. Esta divisón no la considero del todo correcta ya que por un lado no se poseen datos para test y además creo que la proporción no es adecuada ya que desde mi punto de vista está descompensada. Es por eso que más adelante lo que haré será combinar ambos dataset y realizar una nueva división para obtener una proporción más razonable. Sin embargo, lo que si que voy a hacer ahora es combinar ambos datasets en uno para poder explorar los datos y poder determinar si es posible descartar alguna columna.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Extraer los DataFrames de entrenamiento y prueba del dataset cargado
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])

# Combinar los DataFrames de entrenamiento y prueba
dataset = pd.concat([train_df, validation_df], ignore_index=True)

In [ ]:
dataset.shape

(98169, 5)

Combinados ambos datasets, procedo a realizar exploración sobre los datos.

In [ ]:
dataset.dtypes

id          object
title       object
context     object
question    object
answers     object
dtype: object

In [ ]:
dataset.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


Observando el dataser obtenido, se pueden extraer las siguientes conclusiones:

* El primer campo, id, es un campo identificador de cada contexto-pregunta-respuesta. Este campo puede ser eliminado porque para ajustar el modelo no es necesario.

* El segundo campo, title, parece que indica el tema del que trata el contexto-pregunta-respuesta. Sería intersante hacer uso de la función value_counts para comprobar la distribución de los datos en base a este campo.

* El tercer campo, context, es el campo que contiene el texto que se utiliza como contexto para la pregunta. En este caso, se puede observar como para los primeros cinco registros, el contexto es el mismo pero lo que varía entre cada fila es la pregunta y por lo tanto también la respuesta.

* El cuarto campo, question, es un campo de tipo texto libre que contiene la pregunta que se realiza en base al contexto utilizado.

* Por último, el último campo, answer, es un campo tipo diccionario con dos campos: text y answer_start. El primero recoge la respuesta a la pregunta y el segundo parece que indica el número a partir del cual se puede obtener la respuesta en el contexto. Esto lo voy a comprobar más adelante.

Lo primero que voy a hacer es realizar la separación entre train y test para evitar contaminación. Los datos de test se supone que no se deben conocer por lo que es aconsejable guardarlos en ficheros CSV apartes y así evitar confusiones.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Divido los datos y los guardo en ficheros CSV para evitar contaminaciones
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Guardo los conjuntos de datos en archivos CSV
train.to_csv("/content/drive/MyDrive/Práctica LLM/data_train.csv", index=False)
test.to_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv", index=False)


print("Dimensión conjunto de entrenamiento",train.shape)
print("Dimensión conjunto de prueba",test.shape)


Dimensión conjunto de entrenamiento (78535, 5)
Dimensión conjunto de prueba (19634, 5)


# 3. Exploración de los datos

A partir de ahora voy a trabajar con los datos de train. Importo de nuevo el fichero CSV como punto de control.

In [ ]:
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_train.csv")
print(train_data.head())
print("Dimensión conjunto de entrenamiento",train_data.shape)

                         id                                title  \
0  57337343d058e614000b5b32  Financial_crisis_of_2007%E2%80%9308   
1  57267d66f1498d1400e8e180             The_Sun_(United_Kingdom)   
2  5730e7ddf6cb411900e24535                               Tuvalu   
3  5727f92b4b864d19001640fc                   History_of_science   
4  572e92c5c246551400ce436c                    Canadian_football   

                                             context  \
0  This meant that nearly one-third of the U.S. l...   
1  Murdoch found he had such a rapport with Larry...   
2  New Zealand has an annual quota of 75 Tuvaluan...   
3  The scientific revolution is a convenient boun...   
4  In the CFL, if the game is tied at the end of ...   

                                            question  \
0  What institution reported that the traditional...   
1    What position did Larry Lamb take with the Sun?   
2  What is the program that allows season employe...   
3                       What w

Como se ha comentado previamente, es interesante conocer la distribución de las preguntas/respuestas en base al contexto dado. Por ello, compruebo la distribución de los datos en tanto por ciento.

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS
title_counts = train_data.value_counts('title')
total_questions = title_counts.sum()
title_percentage = (title_counts / total_questions) * 100

print("Distribución datos dataset en base al tópico:", title_percentage)


Distribución datos dataset en base al tópico: title
New_York_City            0.848030
Super_Bowl_50            0.825110
American_Idol            0.809830
Beyoncé                  0.762717
Frédéric_Chopin          0.742344
                           ...   
Grape                    0.044566
Great_Plains             0.043293
Pitch_(music)            0.039473
Matter                   0.026740
Myocardial_infarction    0.020373
Name: count, Length: 490, dtype: float64


De un primer vistazo se puede observar como la distribución de los tópicos en el dataset se encuentra descompensado ya que el tópico que más se repita es en de New_York con un 0.84 mientas que preguntas acerca del tópico del miocardio apenas suponen un 0.02 de las preguntas. Esto es interesante conocerlo ya que esto puedo servir para saber hacia donde puede tender el modelo a sesgarse. A la hora de reportar las métricas sobre el rendimiento del modelo, es conveniente conocer los datos con los que se ha entrenado el modelo.

Por otro lado, el campo id y title puede ser eliminado ya que no sirven para ajustar el modelo base. Por ello procedo a su eliminación.

In [ ]:
# Elimino las columnas ID y title
train_data = train_data.drop('title', axis=1)
train_data = train_data.drop('id', axis=1)

In [ ]:
train_data

,context,question,answers
0,This meant that nearly one-third of the U.S. l...,What institution reported that the traditional...,"{'text': ['Brookings Institution'], 'answer_st..."
1,Murdoch found he had such a rapport with Larry...,What position did Larry Lamb take with the Sun?,"{'text': ['editor'], 'answer_start': [160]}"
2,New Zealand has an annual quota of 75 Tuvaluan...,What is the program that allows season employe...,{'text': ['Australian Pacific Seasonal Worker ...
3,The scientific revolution is a convenient boun...,What was Galileo's nickname?,"{'text': ['Father of Modern Physics'], 'answer..."
4,"In the CFL, if the game is tied at the end of ...",Which CFL games require tie-breaking rounds co...,"{'text': ['playoff or championship'], 'answer_..."
...,...,...,...
78530,Adams sent condolences to Donda West's family ...,On what day did the final coroner's report sho...,"{'text': ['January 10, 2008'], 'answer_start':..."
78531,Tuberculosis has been present in humans since ...,In which U.S. state was the oldest definitive ...,"{'text': ['Wyoming'], 'answer_start': [171]}"
78532,Tucson is known for being a trailblazer in vol...,What major cities later adopted Tucson's city ...,"{'text': ['San Francisco and New York City'], ..."
78533,Beyoncé's work has influenced numerous artists...,Which rock band cited Beyonce on their third a...,"{'text': ['White Rabbits'], 'answer_start': [2..."


Por otro lado, investigo el campo answer para comprobar si el valor del campo answer_start corresponde a la posición del caracter a partir de la cual en el contexto se obtiene la respuesta. Para ello divido la columna answers en dos en función de los campos del diccionario.

In [ ]:
# Creo una función para dividir un campo con dos diccionarios en dos columnas
def parse_answers(row):
    answers_dict = eval(row['answers'])
    return pd.Series([answers_dict['text'][0], answers_dict['answer_start'][0]], index=['answers', 'answer_start'])

# Aplico la función a la columna
train_data[['answers', 'answer_start']] = train_data.apply(parse_answers, axis=1)
print(train_data)

                                                 context  \
0      This meant that nearly one-third of the U.S. l...   
1      Murdoch found he had such a rapport with Larry...   
2      New Zealand has an annual quota of 75 Tuvaluan...   
3      The scientific revolution is a convenient boun...   
4      In the CFL, if the game is tied at the end of ...   
...                                                  ...   
78530  Adams sent condolences to Donda West's family ...   
78531  Tuberculosis has been present in humans since ...   
78532  Tucson is known for being a trailblazer in vol...   
78533  Beyoncé's work has influenced numerous artists...   
78534  Despite the position of the official organizat...   

                                                question  \
0      What institution reported that the traditional...   
1        What position did Larry Lamb take with the Sun?   
2      What is the program that allows season employe...   
3                           What was Ga

Compruebo si el valor indicado en la columna answer_start corresponde a la posición del caracter a partir de la cual se obtiene la respuesta en el contexto.

In [ ]:
# Genero índice aleatorio
indice_aleatorio = random.randint(0, train_data.shape[0])

# Imprimo el contexto y la respuesta del campo text
print(f"Contexto: {train_data.loc[indice_aleatorio, 'context']}")
print(f"Respuesta: {train_data.loc[indice_aleatorio, 'answers']}")

# Extraigo la respuesta del contexto usando 'answer_start'
contexto = train_data.loc[indice_aleatorio, 'context']
answer_start = train_data.loc[indice_aleatorio, 'answer_start']
respuesta = train_data.loc[indice_aleatorio, 'answers']

respuesta_extraida = contexto[answer_start:answer_start + len(respuesta)]
print("--------------------------------")
print(f"Respuesta extraída del contexto: {respuesta_extraida}")


Contexto: James Hutton is often viewed as the first modern geologist. In 1785 he presented a paper entitled Theory of the Earth to the Royal Society of Edinburgh. In his paper, he explained his theory that the Earth must be much older than had previously been supposed in order to allow enough time for mountains to be eroded and for sediments to form new rocks at the bottom of the sea, which in turn were raised up to become dry land. Hutton published a two-volume version of his ideas in 1795 (Vol. 1, Vol. 2).
Respuesta: James Hutton
--------------------------------
Respuesta extraída del contexto: James Hutton


Como puede comprobarse, el valor indicado en la columna answer_start corresponde a la posición del caracter a partir de la cual se extrae la solución en el caracter.

Seguidamente, compruebo que no hay valores no nulos o faltantes. En caso de que los haya procedo a eliminarlos del dataset.


In [ ]:
import pandas as pd

# Mostrar el número de valores nulos en cada columna
print("Valores nulos o None antes de la limpieza:")
print(train_data.isnull().sum())

# Eliminar las filas que contienen valores nulos o None en cualquiera de las columnas
train_data = train_data.dropna(subset=['context', 'question', 'answers', 'answer_start'])
print(train_data.shape)

Valores nulos o None antes de la limpieza:
context         0
question        0
answers         0
answer_start    0
dtype: int64
(78535, 4)


En este caso no había valores nulos, pero he querido asegurarme antes de proceder con la separación entre train y validación.

Por último, divido los datos de train en train y validación y guardo los ficheros en CSV para tenerlo como puntos de control.

In [ ]:
# Divido los datos y los guardo en ficheros CSV para evitar contaminaciones
train, validation = train_test_split(train_data, test_size=0.2, random_state=42)

# Guardo los conjuntos de datos en archivos CSV
train.to_csv("/content/drive/MyDrive/Práctica LLM/data_train.csv", index=False)
validation.to_csv("/content/drive/MyDrive/Práctica LLM/data_validation.csv", index=False)


print("Dimensión conjunto de entrenamiento",train.shape)
print("Dimensión conjunto de prueba",validation.shape)

Dimensión conjunto de entrenamiento (62828, 4)
Dimensión conjunto de prueba (15707, 4)


# 4. Pipeline del proyecto

Antes de proceder con el entrenamiento, importo los datos de test e introduzco los mismos cambios generados sobre train.

In [ ]:
import pandas as pd
test_data = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv")
print(test_data.head())
print("Dimensión conjunto de entrenamiento",test_data.shape)

                         id                  title  \
0  572fe03bb2c2fd140056853e         Premier_League   
1  57336eb64776f41900660abf  Saint_Barth%C3%A9lemy   
2  5727a7852ca10214002d9300        Political_party   
3  570d9bebdf2f5219002ed026  Anti-aircraft_warfare   
4  56e76d6537bdd419002c3f93                Teacher   

                                             context  \
0  Television has played a major role in the hist...   
1  International investment and the wealth genera...   
2  In the United Kingdom, it has been alleged tha...   
3  Some nations started rocket research before Wo...   
4  There are many similarities and differences am...   

                                            question  \
0                        What was the cause of this?   
1     St. Barts is considered a playground for whom?   
2  What act did parliament put into place to stop...   
3  Which country's research was ahead of all othe...   
4              Where are nearly all teachers taught?   

 

In [ ]:
# Elimino las columnas
test_data = test_data.drop('title', axis=1)
test_data = test_data.drop('id', axis=1)

# Separo las columnas answers en dos: answers y answers_start
def parse_answers(row):
    answers_dict = eval(row['answers'])
    return pd.Series([answers_dict['text'][0], answers_dict['answer_start'][0]], index=['answers', 'answer_start'])

# Aplico la función a la columna
test_data[['answers', 'answer_start']] = test_data.apply(parse_answers, axis=1)
print(test_data)

# Mostrar el número de valores nulos en cada columna
print("Valores nulos o None antes de la limpieza:")
print(test_data.isnull().sum())

# Elimino las filas que contienen valores nulos o None en cualquiera de las columnas
test_data = test_data.dropna(subset=['context', 'question', 'answers', 'answer_start'])
print(test_data.shape)

                                                 context  \
0      Television has played a major role in the hist...   
1      International investment and the wealth genera...   
2      In the United Kingdom, it has been alleged tha...   
3      Some nations started rocket research before Wo...   
4      There are many similarities and differences am...   
...                                                  ...   
19629  In the First World War, Devonport was the head...   
19630  iPod batteries are not designed to be removed ...   
19631  The landing was north of Sevastopol, so the Ru...   
19632  Pope Leo X was used to reformers and heretics,...   
19633  Frédéric François Chopin (/ˈʃoʊpæn/; French pr...   

                                                question  \
0                            What was the cause of this?   
1         St. Barts is considered a playground for whom?   
2      What act did parliament put into place to stop...   
3      Which country's research was ahe

In [ ]:
# Guardo el fichero con los cambios aplicados

test_data.to_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv", index=False)
print("Dimensión conjunto de prueba",test_data.shape)


Dimensión conjunto de prueba (19634, 4)


# 5. Tokenización

Una vez que los datasets están ya preparados, procedo a tokenizarlos. Es importante tener en cuenta que para el caso de train y validación tanto las preguntas, como el contexto y las respuestas deben ser tokenizados. En cambio, por su parte, para las de tipo test solamente las preguntas y las respuestas deben ser tokenizadas ya que se supone que el modelo no puede ver las respuestas. Por ello la tokenización para test es un poco distinta que para train y validación.

Como punto de control y para no tener que ejecutar todo el código anterior siempre que comience, importo los ficheros CSV generados.

In [ ]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_train.csv")
validation= pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_validation.csv")
test = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv")

print("Dimensión conjunto de entrenamiento",train.shape)
print("Dimensión conjunto de validación",validation.shape)
print("Dimensión conjunto de prueba",test.shape)



Dimensión conjunto de entrenamiento (62828, 4)
Dimensión conjunto de validación (15707, 4)
Dimensión conjunto de prueba (19634, 4)


A continuación convierto los dataframes de pandas a datasets de Hugging Face.

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(validation)
test_dataset = Dataset.from_pandas(test)


Tokenizo el conjunto de train y de validación utilizando el tokenizador del modelo base. Para ello, tomo los datos de las preguntas y contextos, los tokenizo y calculo las posiciones de inicio y fin de las respuestas dentro de los tokens.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    answers = examples["answers"]
    start_chars = examples["answer_start"]

    # Parámetros de la tokenización
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length"
    )
    # Posición inicio y fin
    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = start_chars[i]

        if answer is None or start_char is None:
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)
        # Encuentra el inicio y el fin
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Si la respuesta no está en el contexto, la etiqueto como 0,0
        if not (offset[context_start][0] <= start_char < offset[context_end][1]):
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_idx = None
            end_idx = None

            for idx, (start, end) in enumerate(offset):
                if start <= start_char < end:
                    start_idx = idx
                if start < end_char <= end:
                    end_idx = idx
                    break

            if start_idx is not None and end_idx is not None:
                start_positions.append(start_idx)
                end_positions.append(end_idx)
            else:
                start_positions.append(0)
                end_positions.append(0)

    inputs["start_positions"] = start_positions #Posición inicio
    inputs["end_positions"] = end_positions # Posición fin
    return inputs



Creo la función para tokenizar test. En este caso no se tokeniza la respuesta y por lo tanto tampoco hace falta calcular la posición de inicio/fin de la respuesta.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def preprocess_function_test(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Al ser test, no se tokeniza la respuesta por lo que no hace falta calcular la posición de inicio y fin
    offset_mapping = inputs.pop("offset_mapping")
    inputs["offset_mapping"] = offset_mapping
    return inputs



Tokenizo los tres datasets utilizando las funciones definidas en el apartado anterior.

In [ ]:
# Tokenización
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)
tokenized_test = test_dataset.map(preprocess_function_test, batched=True, remove_columns=test_dataset.column_names)

# Dimensiones
print(tokenized_train.shape)
print(tokenized_validation.shape)
print(tokenized_test.shape)

Map:   0%|          | 0/62828 [00:00<?, ? examples/s]

Map:   0%|          | 0/15707 [00:00<?, ? examples/s]

Map:   0%|          | 0/19634 [00:00<?, ? examples/s]

(62828, 4)
(15707, 4)
(19634, 3)


# 6. Entrenamiento

## 6.1 Primer entrenamiento: Capas sin congelar y usando todo los datos posibles

Cargo el modelo y defino los hiperparámetros del entrenamiento. Como no sé el coste que supone este entrenamiento, selecciono unos hiperparámetros aleatorios para ajustarlos luego.

In [ ]:
from transformers import DistilBertForQuestionAnswering, Trainer, TrainingArguments

# Cargar el modelo preentrenado
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

#Cuantificación
quantized_model = torch.quantization.quantize_dynamic(
model, {torch.nn.Linear}, dtype=torch.qint8)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01)

# Configurar el Trainer
trainer = Trainer(
    model=quantized_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Entreno el modelo

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

Como puede verse realizar este entrenamiento no tiene sentido debido al coste computacional y el tiempo requerido para hacerlo (143 horas aproximadamente). Los hiperparámetros utilizados en el modelo no son altos y aún así el modelo requiere de mucho tiempo para ser entrenado. Con el fin de conseguir una solución, aunque no sea la mejor, voy a reducir los tamaños de los datasets para que así el modelo no requiera procesar tantos datos. Obviamente, lo mejor sería entrenar el modelo con todos los datos disponibles, pero dado que no se puede, entreno con menos aunque eso suponga que no sea tan bueno.

## 6.2 Segundo entrenamiento: Modelo sin congelar usando el 20% de los datos disponibles

In [ ]:
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_train.csv")
validation= pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_validation.csv")
test = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv")

# Reduzco a un 0.2% el tamaño. Selecciono aleatoriamnente los registros con los que nos quedamos.
train = train.sample(frac=0.2, random_state=42)
validation = validation.sample(frac=0.2, random_state=42)
test = test.sample(frac=0.2, random_state=42)

print("Dimensión conjunto de entrenamiento",train.shape)
print("Dimensión conjunto de validación",validation.shape)
print("Dimensión conjunto de prueba",test.shape)

# Creo los datasets de nuevo
train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(validation)
test_dataset = Dataset.from_pandas(test)


Dimensión conjunto de entrenamiento (12566, 4)
Dimensión conjunto de validación (3141, 4)
Dimensión conjunto de prueba (3927, 4)


Tokenizo train, validación y test con las mismas funciones generadas anteriormente.

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)
tokenized_test = test_dataset.map(preprocess_function_test, batched=True, remove_columns=test_dataset.column_names)


Map:   0%|          | 0/12566 [00:00<?, ? examples/s]

Map:   0%|          | 0/3141 [00:00<?, ? examples/s]

Map:   0%|          | 0/3927 [00:00<?, ? examples/s]

Por tener todo más organizado, vuelvo e generar el entrenador y entreno el modelo.

In [ ]:
from transformers import DistilBertForQuestionAnswering, Trainer, TrainingArguments

# Cargo el modelo preentrenado
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

#Cuantificación
quantized_model = torch.quantization.quantize_dynamic(
model, {torch.nn.Linear}, dtype=torch.qint8)

# Hiperparámetros
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Entrenador
trainer = Trainer(
    model=quantized_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Entrenamiento
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

Al haber reducido los tamaños de los datasets en un 20%, el tiempo de entrenamiento ha disminuido pero sigue siendo muy eleveado (25 horas aproximadamente). Esto se debe a que estoy intentando entrenar el modelo desde la base, es decir, estoy intentando entrenar todas las capas del modelo. Como consecuencia, esto aumenta el coste significativamente. Por ello, lo que voy a hacer es congelar todas las capas y entrenar unicamente la última capa. Como consecuencia, el modelo no va a ser tan bueno, pero mi objetivo es el de conseguir un modelo que funcione, aunque sus prestaciones no sean tan buenas.

## 6.3 Tercer entrenamiento: Modelo congelado con un 10% de los datos disponibles

Voy a generar todo de nuevo para tener todo más organizado.En primer lugar, genero las funciones para tokenizar los datasets.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import DistilBertForQuestionAnswering, Trainer, TrainingArguments, DistilBertTokenizer
import torch
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def token_train_valid(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    answers = examples["answers"]
    start_chars = examples["answer_start"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length"
    )
    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = start_chars[i]

        if answer is None or start_char is None:
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        if not (offset[context_start][0] <= start_char < offset[context_end][1]):
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_idx = None
            end_idx = None

            for idx, (start, end) in enumerate(offset):
                if start <= start_char < end:
                    start_idx = idx
                if start < end_char <= end:
                    end_idx = idx
                    break

            if start_idx is not None and end_idx is not None:
                start_positions.append(start_idx)
                end_positions.append(end_idx)
            else:
                start_positions.append(0)
                end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


def token_test(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Al ser test, no se tokeniza la respuesta por lo que no hace falta calcular la posición de inicio y fin
    offset_mapping = inputs.pop("offset_mapping")
    inputs["offset_mapping"] = offset_mapping
    return inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Seguidamente, cargo los datasets. Como puede observarse, solamente utilizo el 10% de los datos debido a que aunque aquí no aparezca, he intentado realiar un entrenamiento con el modelo congelado con un 20% de los datos y el tiempo estimado para realizar el entrenamiento era de 17 horas. Es por eso que reduzco los datos a usar en un 10%.

In [ ]:
# Cargo el dataset
train = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_train.csv")
validation = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_validation.csv")
test = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv")

# Reduzco el tamaño al 10%
train = train.sample(frac=0.1, random_state=42)
validation = validation.sample(frac=0.1, random_state=42)
test = test.sample(frac=0.1, random_state=42)

# Dimensiones de los datasets
print("Dimensión conjunto de entrenamiento reducido:", train.shape)
print("Dimensión conjunto de validación reducido:", validation.shape)
print("Dimensión conjunto de prueba reducido:", test.shape)

# Creo los datasets de nuevo
train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(validation)
test_dataset = Dataset.from_pandas(test)

# Tokenizo train, validación y test
tokenized_train = train_dataset.map(token_train_valid, batched=True)
tokenized_validation = validation_dataset.map(token_train_valid, batched=True)
tokenized_test = test_dataset.map(token_test, batched=True)



Dimensión conjunto de entrenamiento reducido: (6283, 4)
Dimensión conjunto de validación reducido: (1571, 4)
Dimensión conjunto de prueba reducido: (1963, 4)


Map:   0%|          | 0/6283 [00:00<?, ? examples/s]

Map:   0%|          | 0/1571 [00:00<?, ? examples/s]

Map:   0%|          | 0/1963 [00:00<?, ? examples/s]

Finalmente, añado el comando para congelar todas las capas del modelo menos la última. Dado que no sé como se llaman las capas, imprimo sus nombres.

In [ ]:
from transformers import DistilBertForQuestionAnswering

# Cargar el modelo
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Imprimir los nombres de las capas
for name, param in model.named_parameters():
    print(name)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


distilbert.embeddings.word_embeddings.weight
distilbert.embeddings.position_embeddings.weight
distilbert.embeddings.LayerNorm.weight
distilbert.embeddings.LayerNorm.bias
distilbert.transformer.layer.0.attention.q_lin.weight
distilbert.transformer.layer.0.attention.q_lin.bias
distilbert.transformer.layer.0.attention.k_lin.weight
distilbert.transformer.layer.0.attention.k_lin.bias
distilbert.transformer.layer.0.attention.v_lin.weight
distilbert.transformer.layer.0.attention.v_lin.bias
distilbert.transformer.layer.0.attention.out_lin.weight
distilbert.transformer.layer.0.attention.out_lin.bias
distilbert.transformer.layer.0.sa_layer_norm.weight
distilbert.transformer.layer.0.sa_layer_norm.bias
distilbert.transformer.layer.0.ffn.lin1.weight
distilbert.transformer.layer.0.ffn.lin1.bias
distilbert.transformer.layer.0.ffn.lin2.weight
distilbert.transformer.layer.0.ffn.lin2.bias
distilbert.transformer.layer.0.output_layer_norm.weight
distilbert.transformer.layer.0.output_layer_norm.bias
distil

Mi objetivo es entrenar el modelo congelando todas las capas menos la última. Sin embargo, en este caso creo que es más interesante no congelar las dos últimas que son las que ofrecen los pesos y el bias. Por ello, congelo todas aquellas capas que no contengan el nombre distilbert en su nombre.

 Por otro lado, modifico los hiperparámetros del entrenamiento. La diferencia con respecto al anterior son que he disminuido el número de épocas a dos,  he aumentado el tamaño de batch a 32 y he añadido el parámetro de FP16 para reducir el uso de memoria y acelerar los cálculos. Los valores elegidos como hiperparámetos lo he hecho probando distintas configuraciones y observando cual era la que ofrecía el menor tiempo de entrenamiento. Esto lo he hecho para reducir el tiempo de entrenamiento.

In [ ]:
# Cargo el tokenizador y el modelo
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

#Cuantificación
quantized_model = torch.quantization.quantize_dynamic(
model, {torch.nn.Linear}, dtype=torch.qint8)

# Congelo todas las capas excepto la última
for name, param in quantized_model.named_parameters():
    if 'distilbert' not in name:
        param.requires_grad = False

# Hiperparámetros
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True)

# Entrenador
trainer = Trainer(
    model=quantized_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Inicio el entrenamiento
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: quantized::linear_dynamic: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at ../torch/csrc/autograd/autograd_not_implemented_fallback.cpp:63.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,6.001226
2,No log,6.001226


TrainOutput(global_step=394, training_loss=5.992765010310913, metrics={'train_runtime': 13279.4834, 'train_samples_per_second': 0.946, 'train_steps_per_second': 0.03, 'total_flos': 578114813952.0, 'train_loss': 5.992765010310913, 'epoch': 2.0})

Evaluo el rendimiento del modelo en el dataset de validation.

In [ ]:
trainer.evaluate(eval_dataset=tokenized_validation)


{'eval_loss': 6.00122594833374,
 'eval_runtime': 1141.3647,
 'eval_samples_per_second': 1.376,
 'eval_steps_per_second': 0.044,
 'epoch': 2.0}

Por último, dado que en total he requerido de 4 horas para realizar el entrenamiento y su evaluación con respecto al dataset de validación, guardo el modelo para no tener que entrenarlo de nuevo.

In [ ]:
import os
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

# Especifico la ruta de guardado
save_path = "/content/drive/MyDrive/Práctica LLM/modelo guardado"
os.makedirs(save_path, exist_ok=True)

# Guardo el modelo entrenado y el tokenizador en la ruta especificada
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

# Guardo el estado del entrenamiento
trainer.state.save_to_json(os.path.join(save_path, "trainer_state.json"))

print("Modelo y estado guardados correctamente.")




Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo y estado guardados correctamente.


# 7. Post-entrenamiento

Una vez generado el modelo, deseo conocer que tan bueno es el modelo. Por eso, quiero utilizar el modelo con los datos de test (que son datos que no ha visto nunca) y generar una métrica que estime el rendimiento del modelo. Por ello, lo primero cargo el modelo entrenado y el dataset de test.

In [19]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
import pandas as pd
from datasets import load_metric
from tqdm import tqdm

# Cargo el modelo
save_path = "/content/drive/MyDrive/Práctica LLM/modelo guardado"
tokenizer = DistilBertTokenizer.from_pretrained(save_path)
model = DistilBertForQuestionAnswering.from_pretrained(save_path)

print("Modelo y tokenizador cargados correctamente.")


# Cargo el conjunto de datos de prueba. Utilizo solo el 10% de los datos de test también para evaluarlo y que no requiera tanto tiempo.
dataset = pd.read_csv("/content/drive/MyDrive/Práctica LLM/data_test.csv")
test_data = dataset.sample(frac=0.1, random_state=42)
print(test_data.head())
print("Dimensión conjunto de prueba", test_data.shape)


Modelo y tokenizador cargados correctamente.
                                                 context  \
7840   But house was also being developed on Ibiza,[c...   
15241  Players may only be transferred during transfe...   
13349  Throughout history, many rulers, empires and n...   
10781  Following the Ulm Campaign, French forces mana...   
13838  In October 2011, the government declared that ...   

                                                question              answers  \
7840   what was a popular club in ibiza that started ...              Amnesia   
15241  During which time can a player be transferred ...     transfer windows   
13349                   Who led the Spanish Inquisition?  Tomás de Torquemada   
10781          When was the Battle of Austerlitz fought?           2 December   
13838  How large is the Marshall Islands shark sanctu...              772,000   

       answer_start  
7840            251  
15241            39  
13349           412  
10781           808

A continuación, genero la función para evaluar el modelo con los datos de test. Para ello, comparo las respuestas generadas por el modelo con las respuestas del dataset. Asimismo, añado un truncamiento igual a 512 porque sino el modelo no es capaz de procesar tantos tokens.

In [20]:
def evaluate_model(model, tokenizer, dataset, metric):
    model.eval()
    for idx, example in tqdm(dataset.iterrows(), total=len(dataset), desc="Evaluando"):
        question = example['question']
        context = example['context']
        true_answer = example['answers']

        # Trunco las secuencias que exceden la longitud máxima
        inputs = tokenizer.encode_plus(
            question, context,
            return_tensors='pt',
            truncation=True,
            max_length=512
        )
        input_ids = inputs["input_ids"].tolist()[0]

        with torch.no_grad():
            outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1

        predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

        # Convierto a formato requerido por la métrica
        metric.add(prediction={'id': str(idx), 'prediction_text': predicted_answer},
                   reference={'id': str(idx), 'answers': {'answer_start': [0], 'text': [true_answer]}})

    # Calcular y devolver las métricas
    final_score = metric.compute()
    return final_score


Por último, evaluo el modelo.

In [21]:
# Evaluar el modelo
evaluation_results = evaluate_model(model, tokenizer, test_data, metric)
print(evaluation_results)

Evaluando: 100%|██████████| 1963/1963 [10:58<00:00,  2.98it/s]


{'exact_match': 0.0, 'f1': 2.872596628906776}


Las métricas generadas durante el entrenamiento no son muy buenas porque reflejan que ninguna respuesta generada por el modelo coincide con una respuesta del dataset y además el F1, métrica que tiene en cuenta tanto la precisión como la exhaustividad es muy baja. Al ser el valor de F1 tan bajo lo que está sugeriendo es que el modelo no está capturando correctamente las respuestas en base al contexto dado. Por lo tanto, es posible afirmar que el modelo generado no es bueno.

Las posibles causas de haber obtenido este resultado pueden ser varias:

* En primer lugar, debido a falta de recusos computacionales he tenido que congelar todas las capas menos dos y he tenido que disminuir el tamaño de los datasets a un 10%, por lo que el modelo no ha sido entrenado con tantos datos. Esto puede haber provocado que el modelo no haya ajustado correctamente sus pesos haciendo que las predicciones frente a las respuestas originales difieran.

* Por otro lado, para poder evaluar el modelo con los datos de test he tenido que truncar las entradas a 512, lo cual tiene consecuencias negativas ya que puede haber contextos y preguntas con más tokens. Esto puede haber hecho que los resultados no sean tan buenos como los esperados.

* Por último, el fine tuniing realizado se ha generado con hiperparámetros basados en intentar que el tiempo de entrenamiento no fuera tan elevado. Esto puede haber provocado que el fine tunning realizado no sea tan bueno o fino como se podría haber hecho.

De estas tres posibles causas, la única que todavía puede intentar solucionarse sin tener que realizar el fine tunning de nuevo es la segunda. Es por eso que voy a realizar otra evaluación con los datos de test pero dividiendo el contexto en fragmentos más pequeños y evaluar cada fragemento con el fin de comprobar si haciéndolo así, el resultado obtenido es mejor.

In [22]:
def evaluate_model(model, tokenizer, dataset, metric):
    model.eval()
    for idx, example in tqdm(dataset.iterrows(), total=len(dataset), desc="Evaluando"):
        question = example['question']
        context = example['context']
        true_answer = example['answers']

        # Divido el contexto en fragmentos más pequeños
        context_tokens = tokenizer.encode(context, add_special_tokens=False)
        max_length = 512 - len(tokenizer.encode(question, add_special_tokens=True)) - 3  # Espacio para [CLS], [SEP] y [SEP]

        if len(context_tokens) > max_length:
            context_chunks = [context_tokens[i:i + max_length] for i in range(0, len(context_tokens), max_length)]
        else:
            context_chunks = [context_tokens]

        best_prediction = ""
        best_score = 0

        for chunk in context_chunks:
            inputs = tokenizer.encode_plus(
                question,
                chunk,
                return_tensors='pt',
                truncation=True,
                max_length=512
            )
            input_ids = inputs["input_ids"].tolist()[0]

            with torch.no_grad():
                outputs = model(**inputs)
            answer_start = torch.argmax(outputs.start_logits)
            answer_end = torch.argmax(outputs.end_logits) + 1

            predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))


            score = len(predicted_answer)
            if score > best_score:
                best_score = score
                best_prediction = predicted_answer

        # Convierto a formato requerido por la métrica
        metric.add(prediction={'id': str(idx), 'prediction_text': best_prediction},
                   reference={'id': str(idx), 'answers': {'answer_start': [0], 'text': [true_answer]}})

    # Calcular y devolver las métricas
    final_score = metric.compute()
    return final_score

In [23]:
# Evaluo el modelo
evaluation_results = evaluate_model(model, tokenizer, test_data, metric)
print(evaluation_results)

Evaluando: 100%|██████████| 1963/1963 [13:21<00:00,  2.45it/s]


{'exact_match': 0.0, 'f1': 2.9570204088303087}


Implementando la solución de dividir el contexto en fragmentos más pequeños no mejora la solución por lo que podemos afirmar que el modelo generado en esta práctica no es bueno para generar respuestas en base a un contexto y preguntas dadas.

Realizada la práctica, ahora es momento de reflexionar sobre posibles mejoras que se podrían implementar con el fin de obtener una mejor solución.

* Primeramente, creo que sería conveniente realiar una análisis profundo sobre el modelo que se está utilizando y decidir si este es el mejor modelo para realizar el ejercicio planteado. A priori, parece ser que si que podría ser una buena opción pero sería bueno explorar otras alternativas.

* Por otro lado, personalmente, no me ha acabado de convencer la forma en que he realizado la tokenización del dataset. Aunque haya utilizado el tokenizador del modelo, no sé porque creo que la forma en que la he realizado, no es la correcta. Es por eso que intentaría explorar una nueva alternativa de poder hacerlo.

* Asimismo, si tuviera que volver a realizar de nuevo el ejercicio, intentaría asegurarme de poseer un ordenador con más capacidad computacional. Debido a motivos personales, he tenido que realizar la práctica con un ordenador "básico" y creo que eso ha podido afectar a los resultados obtenidos. Igual con un ordenador con más capacidad no tendría que haber limitado el uso de datos de los datasets, podría haber ajustado los hiperparámetos y podría no haber congelado las capas.

* Por último, creo que el fine tunning realizado no se ha realizado correctamente porque se puede apreciar como la pérdida en ambas épocas es la misma que en la evaluación con los datos de validación. Desde mi punto de vista, esto está indicando que el modelo no está aprendiendo y aunque desconozco el motivo, creo que eso influye.

# Agradecimientos

El resultado de la práctica desde luego no ha sido el esperado y aunque personalmente no me quedo satisfecho con el resultado, he querido presentarlo porque he querido mostrar mi esfuerzo por intentar lograr el objetivo. Es cierto que no es lo esperado, pero a mi personlamente me ha servido para comprender conceptos impartidos durantes las clases y sobretodo para indagar más en el campo de las LLM. Es por eso que mi sensación con esta práctica es agriducle dado que por un lado me ha dado la oportunidad para aprender pero por otro lado no he podido materializarlo en una solución precisa.

Finalmente, quisiera agradecer a nuestro profesor Eric por todo el tiempo que ha invertido en transmitirnos todos los conocimentos y por desvelarnos que la inteligencia artificial no es "magia", sino que hay una explicación matemática que la rige. Sin duda alguna ha sido un módulo muy interesante ya que a mi personalmente me ha permitido conocer todas las posibilidades que puede tener la inteligencia artificial en el futuro. Es por eso que quiero agradecer a nuestro profesor por su tiempo y agradezco a la academia KeepCoding por haber introducido esta asignatura en el bootcamp de IA.